
<a id='top'></a>
<div class="list-group" id="list-tab" role="tablist">
<p style="background-color:#154c79;font-family:newtimeroman;color:#FFF9ED;font-size:150%;text-align:center;border-radius:10px 10px;">TABLE OF CONTENTS</p>   
    
* [1. INTRODUCTION](#1)
    
* [2. IMPORTING LIBRARIES](#2)
    
* [3. DATA PREPARATION](#3)
    
* [4. DATA CLEANING](#4)   
    
* [5. ANALYSIS](#5) 
      
* [6. VISUALIZATION](#6)
    
* [7. CONCLUSION](#7)
    


<a id="1"></a>
# <p style="background-color:#154c79;font-family:newtimeroman;color:#FFF9ED;font-size:100%;text-align:center;border-radius:10px 10px;">1. INTRODUCTION</p>
## **1.1 About dataset**
There are two data sets being used for this project
### - Sydney housing price
200,000 Sydney property sales from 2000-2019 scarped from realestate.com.au
#### *Features*
    ** Date: selling date
    ** Id: transaction id
    ** suburb: suburb of property
    ** postalCode: postal code
    ** sellPrice: selling price
    ** bed: number of bedrooms
    ** bath: number of bathrooms
    ** car: number of car spaces
    ** propType: property type
### - Sydney suburb coordinates
#### *Features*
    ** Suburb: suburb name
    ** Latitude
    ** Longitude

## **1.2 Project**

This project focuses on EDA, visualizing data, and analysing housing price in Sydney.

<a id="2"></a>
# <p style="background-color:#154c79;font-family:newtimeroman;color:#FFF9ED;font-size:100%;text-align:center;border-radius:10px 10px;">2. IMPORTING LIBRARIES</p>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import math
# data visualization
import missingno as msno # visualize missing values
import seaborn as sns 
import plotly.graph_objects as go # map
import plotly.express as px 
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.axes as axes
from matplotlib.ticker import ScalarFormatter
from matplotlib.gridspec import GridSpec # multiple plots


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


### STYLING

In [ ]:
# Set styling
plt.rcParams['figure.figsize'] = (10,8) # setting the graph size globally

# Color for consistent visual
light_color = '#496595'
deep_color = '#c6ccd8'

pd.options.display.float_format = '{:,.0f}'.format # float number format

<a id="3"></a>
# <p style="background-color:#154c79;font-family:newtimeroman;color:#FFF9ED;font-size:100%;text-align:center;border-radius:10px 10px;">3. DATA PREPARATION</p>
This section includes **loading data**, **exploring features**

## 3.1 Loading Data

In [ ]:
housing_price = pd.read_csv("../input/sydney-house-prices/SydneyHousePrices.csv")
suburb = pd.read_csv("../input/suburbs-in-sydney-australia/sydney_suburbs.csv")
    
data = housing_price.copy()
print("Loading Sydney housing price data set")
data.head()

In [ ]:
print("Loading Sydney suburb data set")
suburb.info()

## 3.2 Exploring Features

In [ ]:
data.info()
print("\nNumber of data points: {0:,d}\nNumber of features: {1:10,d}".format(data.shape[0],data.shape[1]))

In [ ]:
# show statistics
data.describe().T

In [ ]:
print("Counting missing data for each feature")
data.isnull().sum()

In [ ]:
# Visualizing missing values
print("Visualizing missing data")
msno.bar(data)

In [ ]:
# Counting unique values for each feature of the data set
count = pd.DataFrame()
for i in data.columns:
    l = len(data[i].value_counts())
    new_row = {'feature':i , 'total_unique_values':l}
    count = count.append(new_row, ignore_index=True)
count

In [ ]:
print('Counting records for each unique value per feature')
for i in data.columns:
    print(data[i].value_counts().to_frame())

### *Findings
* 199,504 data points (records), 9 features
* Incorrect datatypes existing [Date] [bed] [car]
* Missing values existing [bed] [car]
* Inconsistent feature name
* Outliers: [sellPrice] [bed] [bath] [car]
* Other things to consider when doing analysis:
    - house is the most sold property type
    - number of car spaces almost range between (1,10), most common is 1 or 2

<a id="4"></a>
# <p style="background-color:#154c79;font-family:newtimeroman;color:#FFF9ED;font-size:100%;text-align:center;border-radius:10px 10px;">4. DATA CLEANING</p>
This section involves:
* Removing property type other than "house" (for this analysis purpose)
* Dropping column [propType] as not needed anymore.
* Renaming column for consistency
* Removing transactions other than Sydney (postal code should start by 2 and have 4 digits)
* Correcting data types: [Date] (object->date), [bed] & [car] (float->int)
* Handling Null values  [bed] [car]: replacing missing values by 0 (zero)
* Handling Outliers: [bed] [bath] [car] [sellPrice]
* Creating new features (columns) for analysis purpose [year] [month] of sales


In [ ]:
# create new dataframe cdf to store cleaned data
# Remove propType other than "house", drop column [propType]
cdf = data.drop(index=(data[data.propType!='house'].index), columns='propType')

# Remove postalCode >=3000 (keep Sydney only)
cdf.drop(cdf[cdf.postalCode >=3000].index, inplace=True)

# Rename column for consistency
cdf.rename(columns={'Date':'date', 'Id':'id'}, inplace=True)

# Correct data types
cdf['date'] = pd.to_datetime(cdf['date'])

# Handling Null values: replace null values with 0 ([bed], [car])
cdf = cdf.fillna(0)
# convert to best possible data types 
cdf = cdf.convert_dtypes()

cdf.info()

In [ ]:
# recheck if missing value exists
cdf.isnull().sum()

### Results  after cleaning
- 170,105 data points left after cleaning
- All data types corrected (1 datetime, 1 string, 8 interger)
- Missing values are replaced by 0

### Handling Outliers
* [bed] [bath] [car]: using histogram to see distribution then to decide dropping records

* [sellPrice]: using InterQuartile Range technique to remve outliers

* #### Check outliers for number of bedrooms, bathrooms and car spaces

In [ ]:
# Visualizing data with different bin size to explore distribution of data.
fig, ax = plt.subplots(5, 3, figsize=(15,10))
cat = ['bed','bath', 'car']
title = ['bedrooms', 'bathrooms', 'car spaces']

for j, c,t in zip(range(3),cat,title):
    for i,s in zip(range(5),range(5,31,5)):
        ax[i,j].hist(c,data=cdf, bins=s, edgecolor="red", color="lightblue")
        ax[i,j].text(10,20000,t + " / bin size=" + str(s) ,fontsize=10)     
        

**Findings**

Most data distributed in range (0,10) for number of bedrooms, (0,5) for number of bathrooms, and (0,10) for number of car spaces.

Those data points that are out of these ranges will be removed for analysis purpose.

In [ ]:
print("\nNumber of data points: {0:,d}\nNumber of features: {1:10,d}".format(data.shape[0],data.shape[1]))

In [ ]:
# Removing outliers [bed] (drop any bed>10)
bef = len(cdf)
cdf.drop(cdf[cdf.bed >10].index, inplace=True)
aft = len(cdf)
print("Number of data points before: {0:,d}\nTotal data points removed: {2:,d}\nAfter removing [bed]: {1:,d}".format(bef, aft, bef-aft))

In [ ]:
# Removing outliers [bath] (drop any [bath]>5)
bef = len(cdf)
cdf.drop(cdf[cdf.bath >5].index, inplace=True)
aft = len(cdf)
print("Number of data points before: {0:,d}\nTotal data points removed: {2:,d}\nAfter removing [bath]: {1:,d}".format(bef, aft, bef-aft))

In [ ]:
# Removing outliers [car] (drop any [car]>10)
bef = len(cdf)
cdf.drop(cdf[cdf.car >10].index, inplace=True)
aft = len(cdf)
print("Number of data points before: {0:,d}\nTotal data points removed: {2:,d}\nAfter removing [car]: {1:,d}".format(bef, aft, bef-aft))

In [ ]:
var = cdf[['bed','bath', 'car']]
for v in var:
    print('Number of sale transactions per unique value for [', v,']')
    print(cdf[v].value_counts().to_frame())

* #### Check outliers for [sellPrice]


In [ ]:
# Scatterplot to visualize house price over period

plt.scatter(cdf.date, cdf.sellPrice, marker='o', c='b',edgecolor='r', alpha=0.5)
plt.title("Sydney housing price over years")
plt.grid()
plt.show()

There are houses that are sold at prices extremely higher than others, these may affect the analysis results.
To decide how to handle the outliers [sellPrice], I will try diffent approaches
- Approach no#1 Calculate percentile to see data range and skewness:  create df named p_cdf 
- Approach no#2: removing data by using Interquantile technique: create df named IQR_cdf

####  Approach no#1: Calculate percentile, new data set p_cdf

In [ ]:
# Calculate percentile of sell price to see data range, and spot outliers if any 
qr = [0.0001,0.0005,0.0009, 0.001, 0.25, 0.5, 0.75, 0.9, 0.999, 0.9999]
print("Percentiles of house price")
for q in qr:
    print("{0:5,.2f} percentile: {1:15,.0f}".format(q*100, cdf.sellPrice.quantile(q)))
l=0.0009
u=0.9999
lq = cdf.sellPrice.quantile(l)
uq = cdf.sellPrice.quantile(u)
print("\n\033[1mFindings\033[0m\n{0}% of Sydney houses price between ${1:,.0f} and ${2:,.0f}".format((u-l)*100, lq, uq))

#####  Action: Now I am going to create a new dataset p_cdf that contains only data falls into this range ($201163 , $19013790)

In [ ]:
# keeping only points between 0.09 - 99.99 percentitle
p_cdf = cdf.loc[(cdf.sellPrice>=lq) & (cdf.sellPrice<=uq),:]
p_cdf[['sellPrice']].describe().T

In [ ]:
# Calculate number of data points removed
bef = len(cdf)
aft = len(p_cdf)
print("Number of data points before: {0:,d}\nTotal data points removed: {2:,d} (or {3:,.2f}%)\nAfter removing [sellPrice] using percentile: {1:,d}"
      .format(bef, aft, bef-aft, (bef-aft)*100/bef))

####  Approach no#2: Using IQR, new data set IQR_cdf
IQR technique will only keep data points in between Q1-1.5*IQR and Q3+1.5*IQR


In [ ]:
# Define function to remove outliers
def remove_outlier_IQR(df):
    Q1=df.quantile(0.25)
    Q3=df.quantile(0.75)
    IQR=Q3-Q1
    df_final=df[~((df<(Q1-1.5*IQR)) | (df>(Q3+1.5*IQR)))]
    return df_final

def return_final(df,col):
    df_outlier_removed=remove_outlier_IQR(col)
    ind_diff=df.index.difference(df_outlier_removed.index)

    for i in range(0, len(ind_diff),1):
        df_final=df.drop([ind_diff[i]])
        df=df_final
    return df

In [ ]:
# create new df named IQR_cdf
IQR_cdf = cdf
# apply return_final function to remove outliers
IQR_cdf=return_final(IQR_cdf, IQR_cdf['sellPrice'])

In [ ]:
IQR_cdf[['sellPrice']].describe().T

In [ ]:
# Calculate number of data points removed
bef = len(cdf)
aft = len(IQR_cdf)
print("Number of data points before: {0:,d}\nTotal data points removed: {2:,d} (or {3:,.2f}%)\nAfter removing [sellPrice] using IQR: {1:,d}"
      .format(bef, aft, bef-aft, (bef-aft)*100/bef))
print("IQR removed {0:,d} data points, roughly {1:,.2f}% of total data.".format(bef-aft,(bef-aft)*100/bef))

In [ ]:
cdf[['sellPrice']].describe().T

In [ ]:
# Using scatterplot to visualize two approaches, compring how data distribute

fig, ax = plt.subplots(3, figsize=(10, 9), constrained_layout = True, sharex=True)

ax[0].scatter(cdf.date, cdf.sellPrice, marker='o', c='b',edgecolor='r', alpha=0.5)
ax[0].set_title("Sydney housing price over years before removing")

ax[1].scatter(p_cdf.date, p_cdf.sellPrice, marker='o', c='cyan',edgecolor='r', alpha=0.5)
ax[1].set_title("Sydney housing price over years using percentile")

ax[2].scatter(IQR_cdf.date, IQR_cdf.sellPrice, marker='o', c='cyan',edgecolor='r', alpha=0.5)
ax[2].set_title("Sydney housing price over years using IQR")

plt.show()

### Results
Based on the above comparision, I decided to remove outliers by percentile, which only removes 0.1% of data points, comparing to IQR, which drops 5.6% of data and still keeps those points with sell price of $1

Finally, data is cleaned, with 169,211 data points remaining. 
Next step, visualization and analysis.

In [ ]:
# copy cleaned data to overwrite cdf
cdf = p_cdf
cdf.describe().T

<a id="5"></a>
# <p style="background-color:#154c79;font-family:newtimeroman;color:#FFF9ED;font-size:100%;text-align:center;border-radius:10px 10px;">5. ANALYSIS</p>

In this step, data will be aggregated to explore sell price and number of sale transactions on yearly or monthly basis. 
Also, to find out how the housing price relate to location, as well as house's conditions.
To answer following questions:
1. The most expensive locations by suburb and by postal code
2. The busiest month/year for housing market


In [ ]:
# Create new features [year], [month] from [date]
cdf['year'] = pd.DatetimeIndex(cdf['date']).year
cdf['month'] = pd.DatetimeIndex(cdf['date']).month
cdf['month_name'] = cdf['date'].dt.month_name().str[:3]

In [ ]:
# SUBURB: median sell price and number of sale transactions group by suburb, order by median price desc.
sb_df = cdf.groupby('suburb').agg({'sellPrice':'median', 'id':'count'}).reset_index().sort_values(by='sellPrice', ascending=False)
sb_df.columns = ['suburb','medianPrice','transactions']
sb_df

In [ ]:
# Top 10 suburbs with highest median housing price
sb_t10 = sb_df[:10]
sb_t10['color'] = light_color
sb_t10['color'][3:] = deep_color
sb_t10

In [ ]:
# Top 10 suburbs with lowest median price

sb_b10 = cdf.groupby('suburb').agg({'sellPrice':'median', 'id':'count'}).reset_index().sort_values(by='sellPrice', ascending=True)[:10]
sb_b10.columns = ['suburb','medianPrice','transactions']
sb_b10['color'] = light_color
sb_b10['color'][3:] = deep_color
sb_b10

In [ ]:
# POSTAL CODE: median sell price and number of sale transactions group by postal code, order by median price desc.
pc_df = cdf.groupby('postalCode').agg({'sellPrice':'median', 'id':'count'}).reset_index().sort_values(by='sellPrice', ascending=False)
pc_df.columns = ['postalCode','medianPrice', 'transactions']
pc_df

In [ ]:
# Top 10 postal code with highest median housing price
pc_t10 = pc_df[:10]
pc_t10['color'] = light_color
pc_t10['color'][3:] = deep_color
pc_t10

### 1. The most expensive locations to live in Sydney 

In [ ]:
# Top 10 suburbs
ax1=plt.subplot(1,2,1)
y_pos = np.arange(len(sb_t10['suburb'])) # y label locations
ax1.barh(y_pos, sb_t10['medianPrice'],align='center',color=sb_t10['color'])
ax1.set_yticks(y_pos)
ax1.set_yticklabels(sb_t10['suburb'])
ax1.invert_yaxis()  # labels read top-to-bottom
ax1.set_xlabel('Median Sell Price')
ax1.set_title("10 most expensive suburbs", fontdict={'fontsize':14, 'color':'b'})
ax1.grid(axis='x', color='grey', linestyle=':', linewidth=0.5)

# Top 10 postal codes
ax2= plt.subplot(1,2,2)
y_pos = np.arange(len(pc_t10['postalCode'])) # y label locations
ax2.barh(y_pos, pc_t10['medianPrice'],align='center', color=pc_t10['color'])
ax2.set_yticks(y_pos)
ax2.set_yticklabels(pc_t10['postalCode'])
ax2.invert_yaxis()  
ax2.set_xlabel('Median Sell Price')
ax2.set_title("10 most expensive locations by postal code", fontdict={'fontsize':14, 'color':'b'} )
ax2.grid(axis='x', color='grey', linestyle=':', linewidth=0.5)

plt.show()

### 2. The busiest month / year for housing market in Sydney 

In [ ]:
# Grouping data on monthly basis, order by count(id) or number of transactions

month_df = cdf.groupby(['month','month_name']).agg({'id':'count','sellPrice':'median'}).reset_index().sort_values(by='id', ascending=False)
month_df.columns = ['month','month_name','transactions','medianPrice']
month_df

In [ ]:
# Grouping data on monthly basis, no sorting
month2_df = cdf.groupby(['month','month_name']).agg({'id':'count','sellPrice':'median'}).reset_index()
month2_df.columns = ['month','month_name','transactions','medianPrice']
month2_df

In [ ]:
# Grouping data on yearly basis, order by transactions desc
year_df = cdf.groupby('year').agg({'id':'count','sellPrice':'median'}).reset_index().sort_values(by='id', ascending=False)
year_df.columns = ['year', 'transactions','medianPrice']
year_df.head()

In [ ]:
# Grouping data on yearly basis, no sorting
year2_df = cdf.groupby('year').agg({'id':'count','sellPrice':'median'}).reset_index()
year2_df.columns = ['year', 'transactions', 'medianPrice']
year2_df

In [ ]:
# Visualizing number of houses sold per month and per year
plt.figure(figsize=(15, 12))
# monthly volume
month_list = range(math.floor(month_df['month'].min()), math.ceil(month_df['month'].max())+1) #show xticklabel as int
month_df['color'] = light_color
month_df['color'][3:] = deep_color
ax1 = plt.subplot(2,1,1)
ax1.bar('month', 'transactions', data=month_df, color=month_df['color'])
ax1.set_ylabel('Transactions')
ax1.set_xticks(month_list)
ax1.set_title("The busiest month of Sydney housing market", fontdict={'fontsize':14, 'color':'b'})
ax1.grid(axis='y', color='grey', linestyle=':', linewidth=0.5)

# Yearly volume
year_list = range(math.floor(year_df['year'].min()), math.ceil(year_df['year'].max())+1) #show xticklabel as int
year_df['color'] = light_color
year_df['color'][3:] = deep_color

ax2=plt.subplot(2,1,2)
ax2.bar('year', 'transactions', data=year_df, color=year_df['color']) #bar chart
ax2.plot(year2_df['year'], year2_df['transactions'], color='r', marker='o')
ax2.set_xticks(year_list)
ax2.set_ylabel('Transactions')
ax2.set_title("Sydney housing market over years", fontdict={'fontsize':14, 'color':'b'})
ax2.grid(axis='y', color='grey', linestyle=':', linewidth=0.5)
plt.show()

### **Findings**
Sydney's housing market appeared to be busiest in March, November, May and be slowest in January.

From 2000 to 2019, Sydney housing market reached its peak , 23,865 houses being sold. Market has been very active between 2015 and 2017, more than 22,000 transactions each year. 
Then 2019, Australian property market crashed, experienced a sharp fall.

In [ ]:
### Merge two data sets 
# Merge with suburb data set to plot on map
merge_df = pd.merge(sb_df, suburb, how='inner', left_on='suburb', right_on='Suburb')
merge_df.drop(columns='Suburb', inplace=True)
merge_df

<a id="6"></a>
# <p style="background-color:#154c79;font-family:newtimeroman;color:#FFF9ED;font-size:100%;text-align:center;border-radius:10px 10px;">6. VISUALIZATION</p>

In [ ]:
# Dashboard
fig = make_subplots(rows=6, cols=2, vertical_spacing=0.07,
                    specs=[[{'type':'bar'}, {'type':'bar'}], 
                           [{'type':'bar', 'colspan':2}, None],
                           [{'type':'bar', 'colspan':2}, None],
                           [{'type':'Scattergeo', 'rowspan':2,'colspan':2}, None], 
                           [None,None]
                           [{'type':'bar', 'colspan':2}, None]], 
                    
                    
                   subplot_titles=("10 most expensive suburbs", "10 most affordable suburbs",
                                   "Sydney housing market over years", 
                                   "Busiest month of the year",
                                   "Mapping Sydney suburbs"))

fig.add_trace(go.Bar(y=sb_t10['suburb'], x=sb_t10['medianPrice'], #Top 10 suburbs
                     marker=dict(color= sb_t10['color']), 
                     hovertemplate="Suburb: %{y}<br>Median price: %{x}<extra></extra>",
                     name='suburb', orientation='h'),
              row=1,col=1)
fig.add_trace(go.Bar(y=sb_b10['suburb'], x=sb_b10['medianPrice'], # Bottom 10 suburbs
                     marker=dict(color= sb_b10['color']), 
                     hovertemplate="Suburb: %{y}<br>Median price: %{x}<extra></extra>",
                     name='suburb', orientation='h'),
              row=1,col=2)
   
fig.add_trace(go.Scatter(x=year2_df['year'], y=year2_df['transactions'],  # Yearly volume transaction
                         mode='markers+lines', marker=dict(color= 'green'), 
                         hovertemplate="Year: %{x}<br>Number of transactions: %{y}",
                         name='yearly volume'),
              row=2,col=1)
fig.add_trace(go.Bar(x=year_df['year'], y=year_df['transactions'],  # Yearly volume transaction
                     marker=dict(color= year_df['color']), name='yearly volume',
                     hovertemplate="Year: %{x}<br>Number of transactions: %{y}"),
              row=2,col=1)

fig.add_trace(go.Bar(x=month_df['month'], y=month_df['transactions'], # Monthly volume
                     marker=dict(color= month_df['color']),name='monthly volume'), 
              row=3,col=1)

fig.add_trace(go.Scattergeo( lat=merge_df['Latitude'], lon=merge_df['Longitude'],
                            customdata=merge_df['transactions'],
                            hoverinfo='text', hovertext=merge_df['suburb'],
                            hovertemplate="Suburb: %{hovertext}<br>Median price: %{text}<br>Number of transactions: %{customdata}",
                            mode="markers", name='suburb', text=merge_df['medianPrice'],
                            textfont={"color": ["grey"], "family": ["Arial"],"size": [12]},
                            textposition=["top center"],
                            marker={'size':8, 'color': 'blue', 'opacity':0.7,'line':dict(width=1, color='grey')}),
              row=4, col=1)

fig.update_geos(resolution=110, visible=False,  # update map 
                showcountries=True, countrycolor="Black",
                showsubunits=True, showocean=True, 
                subunitcolor="Cyan",showland = True, fitbounds="locations")
fig.update_layout(height=1200, width=1000, 
                  showlegend=False, template="plotly_white", 
                  title_font=dict(size=29, color='#8a8d93', family="Lato, sans-serif"),)
fig.update_yaxes( ticksuffix=' ', categoryorder='total ascending', row=1) # show suburbs by median price order
fig.update_xaxes(tickmode = 'array',tickvals=month_df['month'], ticktext=[i for i in month_df['month_name']], visible=True, row=3, col=1)


<a id="7"></a>

# <p style="background-color:#154c79;font-family:newtimeroman;color:#FFF9ED;font-size:100%;text-align:center;border-radius:10px 10px;">7. CONCLUSION</p>

This project is just for purpose of practising my Python skill, the data set is not fully represented Sydney housing market obviously.

Welcome any feedback or question. Thank you!